In [1]:
%matplotlib inline

In [2]:
from __future__ import annotations

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np

import pyvista as pv
from pyvista import examples
import pyvistaqt as pvqt

np.set_printoptions(linewidth=500)

In [3]:
N_COLUMNS=12
N_ROWS=21

data = np.zeros((N_ROWS, N_COLUMNS))

data[5][5] = 100

data[5][6] = 100
data[5][7] = 100

data[10][5] = 100

data[8][10] = 100



data

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0., 100., 100., 100.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 100.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0., 100.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   

In [50]:
pl = pv.Plotter()

points = np.load("./assets/boob_grid_12_21.npy")

mesh = pv.read('./assets/boob.obj')
mesh.translate(np.array([0, -0.035, 0]), inplace=True)

# Remove points skipped (at 0, 0, 0))
points_not_zero = points[np.sum(points, axis=1) != 0]

point_cloud = pv.PolyData(points)
pl.add_mesh(points_not_zero, color='red')
pl.add_mesh(mesh, opacity=0.5)

pl.show()




Widget(value='<iframe src="http://localhost:59620/index.html?ui=P_0x30e1e4e60_44&reconnect=auto" class="pyvist…

In [51]:
# mesh = pv.read('./boob_mold.obj')
# tex = pv.read_texture('./texture-2.jpg')

# axes = pv.Axes(show_actor=True, actor_scale=2.0, line_width=5)
# axes.origin = (3.0, 3.0, 3.0)


# point_cloud_scaled = point_cloud.scale(1000)

# point_cloud_scaled.rotate_y(150, point=axes.origin, inplace=True)
# point_cloud_scaled.rotate_x(-5, point=axes.origin, inplace=True)

# point_cloud_scaled.translate(np.array([-20, 105, -10]), inplace=True)
# pl = pv.Plotter()

# pl.add_actor(axes.actor)

# axes = pv.Axes(show_actor=True, actor_scale=2.0, line_width=5)
# axes.origin = (3.0, 3.0, 3.0)
# pl.add_mesh(mesh, opacity=0.5)
# pl.add_mesh(point_cloud_scaled, color='red')
# pl.show()


In [52]:
from scipy.spatial import KDTree


k = 5

kdtree = KDTree(points)

mesh['nearest_point'] = kdtree.query(mesh.points)[1]
mesh['nearest_points'] = kdtree.query(mesh.points, k=k)[1]
mesh['nearest_points_dist'] = kdtree.query(mesh.points, k=k)[0]

data_flatten = data.flatten()

# Step 1: Calculate the reciprocal of the distances
reciprocal_distances = 1 / mesh['nearest_points_dist']

# Step 2: Square the reciprocal distances
squared_reciprocal_distances = reciprocal_distances ** 1.2

# Step 3: Normalize the weights
normalized_weights = squared_reciprocal_distances / np.sum(squared_reciprocal_distances, axis=1, keepdims=True)


In [76]:


# Step 4: Apply the weights to compute the weighted sum
point_cloud['pressure'] = data_flatten
mesh['pressure'] = np.sum(data_flatten[mesh['nearest_points']] * normalized_weights, axis=1)

# apple['pressure'] = np.where(apple['pressure'] > 50, 50, apple['pressure'])

pl = pvqt.BackgroundPlotter(shape=(1, 2), border=False)
pl.subplot(0, 0)
pl.add_text("Measured Pressure Values on Grid", font_size=12)
pl.add_mesh(point_cloud, scalars='pressure', cmap='cool', point_size=15)

pl.subplot(0, 1)
pl.add_text(f"Inverse Distance Weighted Interpolation (k={k})", font_size=12)
pl.add_mesh(mesh, scalars='pressure', cmap='cool')

pl.link_views()


pl.show()

: 

In [ ]:

pl = pvqt.BackgroundPlotter()
pl.add_mesh(mesh, scalars='pressure', cmap='RdPu')

for point in points:
    sphere = pv.Sphere(center=point, radius=0.0005)
    pl.add_mesh(sphere, style='points', color='blue')


while True:

    data_flatten = np.roll(data_flatten, 1)
    # Step 4: Apply the weights to compute the weighted sum
    mesh['pressure'] = np.sum(data_flatten[mesh['nearest_points']] * normalized_weights, axis=1)

    # apple['pressure'] = np.where(apple['pressure'] > 50, 50, apple['pressure'])
    pl.render()
    pl.app.processEvents()
